In [1]:
from pathlib import Path
import os, sys

# ---- 프로젝트 루트 & sys.path 설정 ----
PROJECT_ROOT = Path("/workspace/baseball_pipeline").resolve()
os.chdir(PROJECT_ROOT)
sys.path.append(str(PROJECT_ROOT))  # src 패키지 임포트용

print("PROJECT_ROOT:", PROJECT_ROOT)

# ---- 데이터 디렉토리 ----
DATA_DIR = PROJECT_ROOT / "data"
INPUT_VIDEO_DIR = DATA_DIR / "input_videos"
STT_RAW_DIR = DATA_DIR / "stt_raw"
STT_SEG_DIR = DATA_DIR / "stt_segments"
LLM_OUT_DIR = DATA_DIR / "llm_outputs"
FAISS_DIR = PROJECT_ROOT / "faiss_index"
SRC_ROOT = PROJECT_ROOT / "src"


if str(SRC_ROOT) not in sys.path:
    sys.path.append(str(SRC_ROOT))

print("sys.path 추가됨:", SRC_ROOT)


for d in (DATA_DIR, INPUT_VIDEO_DIR, STT_RAW_DIR, STT_SEG_DIR, LLM_OUT_DIR, FAISS_DIR, SRC_ROOT):
    d.mkdir(parents=True, exist_ok=True)

print("INPUT_VIDEO_DIR:", INPUT_VIDEO_DIR) 
print("STT_RAW_DIR    :", STT_RAW_DIR)
print("STT_SEG_DIR    :", STT_SEG_DIR)
print("LLM_OUT_DIR :", LLM_OUT_DIR)
print("FAISS_DIR      :", FAISS_DIR)
print("SRC_ROOT      :", SRC_ROOT)

# ---- CLOVA 토큰 (하드코딩) ----
CLOVA_INVOKE_URL = ""  # 네 invoke URL
CLOVA_SECRET_KEY = ""                                       # 네 secret

# (선택) 다른 토큰들
HF_TOKEN = ""
OPENAI_API_KEY = ""

if HF_TOKEN and "xxx" not in HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
    os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ 기본 경로/토큰 설정 완료")


PROJECT_ROOT: /workspace/baseball_pipeline
sys.path 추가됨: /workspace/baseball_pipeline/src
INPUT_VIDEO_DIR: /workspace/baseball_pipeline/data/input_videos
STT_RAW_DIR    : /workspace/baseball_pipeline/data/stt_raw
STT_SEG_DIR    : /workspace/baseball_pipeline/data/stt_segments
LLM_OUT_DIR : /workspace/baseball_pipeline/data/llm_outputs
FAISS_DIR      : /workspace/baseball_pipeline/faiss_index
SRC_ROOT      : /workspace/baseball_pipeline/src
✅ 기본 경로/토큰 설정 완료


In [3]:
# [셀 1] STT 모듈에서 run_stt_pipeline만 사용해보기

from src.stt_pipeline import run_stt_pipeline

VIDEO_NAME = "clip.mp4"
video_path = INPUT_VIDEO_DIR / VIDEO_NAME

print("video_path:", video_path, "| exists:", video_path.exists())
if not video_path.exists():
    raise FileNotFoundError("data/input_videos/ 안에 clip.mp4 를 넣어주거나 VIDEO_NAME을 변경해줘.")

# (선택) 키워드 엑셀
STT_KEYWORD_XLSX = PROJECT_ROOT / "stt.xlsx"
if STT_KEYWORD_XLSX.exists():
    xlsx_path = STT_KEYWORD_XLSX
    use_domain = False
    print("[STT] stt.xlsx 사용 (엑셀 부스팅)")
else:
    xlsx_path = None
    use_domain = True
    print("[STT] stt.xlsx 없음 → 도메인 키워드 부스팅만 사용")

tts_csv_path, timeline_json_path = run_stt_pipeline(
    audio_path=video_path,
    invoke_url=CLOVA_INVOKE_URL,
    secret_key=CLOVA_SECRET_KEY,
    stt_raw_dir=STT_RAW_DIR,
    stt_seg_dir=STT_SEG_DIR,
    xlsx_keywords_path=xlsx_path,
    use_domain_boostings=use_domain,
    speaker_count_min=2,
    speaker_count_max=3,
    save_raw_json=True,
)

tts_csv_path, timeline_json_path


video_path: /workspace/baseball_pipeline/data/input_videos/clip.mp4 | exists: True
[STT] stt.xlsx 사용 (엑셀 부스팅)
[STT_PIPELINE] Clova STT 요청 시작: /workspace/baseball_pipeline/data/input_videos/clip.mp4
[STT_PIPELINE] raw JSON 저장 -> /workspace/baseball_pipeline/data/stt_raw/clip.clova_raw.json
[STT_PIPELINE] phrase CSV 저장 -> /workspace/baseball_pipeline/data/stt_segments/clip.tts_phrases.csv (rows=50)
[STT_PIPELINE] timeline JSON 저장 -> /workspace/baseball_pipeline/data/stt_segments/clip.timeline.json


(PosixPath('/workspace/baseball_pipeline/data/stt_segments/clip.tts_phrases.csv'),
 PosixPath('/workspace/baseball_pipeline/data/stt_segments/clip.timeline.json'))

In [4]:
# [셀: LLM 단계 테스트]

from src.llm_kanana import run_llm_pipeline

# STT에서 만들어진 CSV 경로
TTS_CSV = STT_SEG_DIR / "clip.tts_phrases.csv"

# 우리가 FAISS 인덱스를 만들 때 사용한 경기 제목
MATCH_TITLE = "2025 준플레이오프 2차전"

out_csv = run_llm_pipeline(
    tts_csv_path=TTS_CSV,
    faiss_dir=FAISS_DIR,
    match_title=MATCH_TITLE,
    model_name="SeHee8546/kanana-1.5-8b-pakchanho-lora",  # 나중엔 파인튜닝 모델 이름으로 교체
    llm_output_dir=LLM_OUT_DIR,
)

out_csv


/workspace/miniconda3/envs/baseball_pipe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[LLM] 입력 CSV: /workspace/baseball_pipeline/data/stt_segments/clip.tts_phrases.csv
[RAG] FAISS 로딩: /workspace/baseball_pipeline/faiss_index
[RAG] 타겟 경기 제목: '2025 준플레이오프 2차전' → 정규화: '2025준플레이오프2차전'
[RAG] 매칭된 조각 수: 3 → 하나로 합칩니다.
[LLM] 모델(혹은 LoRA 어댑터) 로딩 시도: SeHee8546/kanana-1.5-8b-pakchanho-lora
[LLM] LoRA 어댑터로 인식됨 → base_model = kakaocorp/kanana-1.5-8b-instruct-2505


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [01:11<00:00, 17.95s/it]


[LLM] base + LoRA 로딩 완료
[LLM] analyst 행에 대해 박찬호 해설 생성 시작
[LLM] idx=6 (sorted i=6) → analyst 멘트 생성 중...
    - orig: 세터 쪽에 안타가 됩니다. 주자는 3로 돌았어요. 강민호는 홈으로
    - prev caster: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는데 이번 가을은 선취점을 낸 팀이 모두 승리를 한 바 있습니다. 잘 받아친 타구
[LLM] idx=7 (sorted i=7) → analyst 멘트 생성 중...
    - orig: 들어옵니다.
    - prev caster: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는데 이번 가을은 선취점을 낸 팀이 모두 승리를 한 바 있습니다. 잘 받아친 타구
[LLM] idx=8 (sorted i=8) → analyst 멘트 생성 중...
    - orig: 서치점 가져가는 삼성 라이온
    - prev caster: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는데 이번 가을은 선취점을 낸 팀이 모두 승리를 한 바 있습니다. 잘 받아친 타구
[LLM] idx=9 (sorted i=9) → analyst 멘트 생성 중...
    - orig: 김지찬이
    - prev caster: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는데 이번 가을은 선취점을 낸 팀이 모두 승리를 한 바 있습니다. 잘 받아친 타구
[LLM] idx=10 (sorted i=10) → analyst 멘트 생성 중...
    - orig: 이렇게 팀에서 지점을 만들어 냅니다.
    - prev caster: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는데 이번 가을은 선취점을 낸 팀이 모두 승리를 한 바 있습니다. 잘 받아친 타구
[LLM] idx=11 (sorted i=11) → analyst 멘트 생성 중...
    - o

PosixPath('/workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv')

In [3]:
# fish-audio import click 문제
import sys

print("sys.executable:", sys.executable)

# 이 파이썬 해석기 기준으로 패키지 설치
!{sys.executable} -m pip install click hydra-core omegaconf loguru torchaudio


sys.executable: /workspace/miniconda3/envs/baseball_pipe/bin/python


In [1]:
# fish-audio api를 이용한 생성 효율화
import sys

!{sys.executable} -m pip install -U click fastapi uvicorn "huggingface_hub[cli]"


  Attempting uninstall: huggingface_hub━━━━━━━━━ 0/2 [typer-slim]
    Found existing installation: huggingface-hub 0.36.0/2 [typer-slim]
    Uninstalling huggingface-hub-0.36.0:0m━━━━━━━━━━━━━━━━━━━ 1/2 [huggingface_hub]
      Successfully uninstalled huggingface-hub-0.36.0━━━━━━━━━ 1/2 [huggingface_hub]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [huggingface_hub] [huggingface_hub]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.1 requires huggingface-hub<1.0,>=0.34.0, but you have huggingface-hub 1.1.5 which is incompatible.


In [7]:
import sys
!{sys.executable} -m pip install ormsgpack

In [ ]:
# [셀: TTS 단계 테스트]
from pathlib import Path
from src.tts_fishspeech import run_tts_batch

# 이미 셀 1에서 설정해 둔 값들 재사용
# PROJECT_ROOT, DATA_DIR, LLM_OUT_DIR, 등등

# 1) 어떤 영상 기준인지
VIDEO_NAME = "clip.mp4"                    # 필요하면 다른 이름으로 교체
VIDEO_STEM = Path(VIDEO_NAME).stem         # "clip"

# 2) LLM이 만들어 놓은 CSV 경로
llm_csv = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
print("LLM CSV:", llm_csv, "| exists:", llm_csv.exists())
if not llm_csv.exists():
    raise FileNotFoundError("LLM CSV가 없습니다. 파일 경로/이름을 확인해주세요.")

# 3) 캐스터/해설 참조 음성 (data/tts_refs 아래)
TTS_REF_DIR = DATA_DIR / "tts_refs"
caster_ref = TTS_REF_DIR / "caster_prompt.wav"
analyst_ref = TTS_REF_DIR / "analyst_pakchanho_prompt.wav"

print("caster_ref :", caster_ref,  "| exists:", caster_ref.exists())
print("analyst_ref:", analyst_ref, "| exists:", analyst_ref.exists())

if not caster_ref.exists() or not analyst_ref.exists():
    raise FileNotFoundError("참조 음성 wav가 없습니다. data/tts_refs 폴더를 확인해주세요.")

# 4) 배치 TTS 실행 (GPU에서 Fish-Speech 호출)
out_csv = run_tts_batch(
    tts_csv_path=llm_csv,
    caster_ref_wav=caster_ref,
    analyst_ref_wav=analyst_ref,
    # max_rows=5,   # 테스트로 몇 줄만 돌려보고 싶으면 주석 해제
)

print("TTS 결과 CSV:", out_csv)
out_csv



LLM CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv | exists: True
caster_ref : /workspace/baseball_pipeline/data/tts_refs/caster_prompt.wav | exists: True
analyst_ref: /workspace/baseball_pipeline/data/tts_refs/analyst_pakchanho_prompt.wav | exists: True
[TTS_BATCH] 입력 CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv
[TTS_BATCH] video_stem 추론: clip
[TTS_BATCH] TTS 출력 루트: /workspace/baseball_pipeline/data/tts_audio/clip

[TTS_BATCH] (1/50) role=caster utterance_id=clip_1_1
[TTS] prompt_tokens 존재: /workspace/baseball_pipeline/data/tts_refs/caster_prompt.npy
[TTS] Text→Codes: "연속 탈삼진..."

[CMD] /workspace/miniconda3/envs/baseball_pipe/bin/python fish_speech/models/text2semantic/inference.py --text 연속 탈삼진 --prompt-text 캐스터 프롬프트 음성입니다. --prompt-tokens /workspace/baseball_pipeline/data/tts_refs/caster_prompt.npy --checkpoint-path /workspace/baseball_pipeline/fish-speech/checkpoints/openaudio-s1-mini --num-samples 1 --outpu

In [9]:
# [셀: api를 이용한 TTS 단계 테스트]
from pathlib import Path
from src import DATA_DIR
from src.tts_fishspeech_api import run_tts_batch_via_api

LLM_OUT_DIR = DATA_DIR / "llm_outputs"
TTS_REF_DIR = DATA_DIR / "tts_refs"

VIDEO_NAME = "clip.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
caster_ref = TTS_REF_DIR / "caster_prompt.wav"
analyst_ref = TTS_REF_DIR / "analyst_pakchanho_prompt.wav"

out_csv = run_tts_batch_via_api(
    tts_csv_path=llm_csv,
    caster_ref_wav=caster_ref,
    analyst_ref_wav=analyst_ref,
    api_url="http://127.0.0.1:8080/v1/tts",
    # max_rows=5,  # 일부만 먼저 돌려보고 싶으면 켜기
)

out_csv


[TTS_API] 입력 CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv
[TTS_API] video_stem: clip
[TTS_API] 출력 디렉토리: /workspace/baseball_pipeline/data/tts_audio/clip
[TTS_API] (0) role=caster utterance_id=clip_1_1 text='연속 탈삼진...'
[TTS_API] (1) role=caster utterance_id=clip_1_2 text='풀 카운트 승부...'
[TTS_API] (2) role=caster utterance_id=clip_1_3 text='볼렛...'
[TTS_API] (3) role=caster utterance_id=clip_2_1 text='계속되는 풀 카운트 승부 일사주자는 ...'
[TTS_API] (4) role=caster utterance_id=clip_2_2 text='바깥쪽 골렛 연속 골렛이 나옵니다....'
[TTS_API] (5) role=caster utterance_id=clip_3_1 text='테이블 세터진 김지찬, 김성윤 이 두...'
[TTS_API] (6) role=analyst utterance_id=clip_4_1 text='- 문법적으로 자연스러운 문장이어야 ...'
[TTS_API] (7) role=analyst utterance_id=clip_4_2 text='---...'
[TTS_API] (8) role=analyst utterance_id=clip_4_3 text='- 따로 입력된 추가 요청 없이, 주...'
[TTS_API] (9) role=analyst utterance_id=clip_5_1 text='- "빠른 공을 던질 때는 어깨 힘이...'
[TTS_API] (10) role=analyst utterance_id=clip_5_2 text='---...'
[TTS_API] (11

KeyboardInterrupt: 

In [6]:
# [셀: TTS 타임라인 오디오 + 최종 영상 만들기 (TTS만) api 사용]

from pathlib import Path

# ❌ from src import DATA_DIR, INPUT_VIDEO_DIR  <-- 이건 빼고
from src.audio_timeline import build_tts_audio_timeline, mux_tts_audio_to_video

# 셀 1에서 이미 정의한 전역:
# PROJECT_ROOT, DATA_DIR, INPUT_VIDEO_DIR 가 있다고 가정

VIDEO_NAME = "clip.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

video_path = INPUT_VIDEO_DIR / VIDEO_NAME

# TTS API 배치 단계에서 만든 CSV (파일명은 네가 실제 만든 이름으로)
TTS_AUDIO_DIR = DATA_DIR / "tts_audio" / VIDEO_STEM
tts_csv = TTS_AUDIO_DIR / f"{VIDEO_STEM}.tts_phrases.with_tts_api.csv"
print("tts_csv:", tts_csv, "| exists:", tts_csv.exists())

# 1) 원본은 음소거 + TTS 구간만 소리 나는 오디오 타임라인 만들기
mixed_audio_path = TTS_AUDIO_DIR / f"{VIDEO_STEM}.tts_only.wav"
build_tts_audio_timeline(
    video_path=video_path,
    tts_csv_path=tts_csv,
    output_audio_path=mixed_audio_path,
    sample_rate=48000,
    keep_original_gaps=False,   # ✅ 여기! 원본 음성 대신 무음
)

# 2) 이 오디오를 영상에 입혀서 최종 mp4 생성
OUTPUT_VIDEO_DIR = DATA_DIR / "output_videos"
OUTPUT_VIDEO_DIR.mkdir(parents=True, exist_ok=True)

out_video = OUTPUT_VIDEO_DIR / f"{VIDEO_STEM}.tts_only.mp4"
mux_tts_audio_to_video(
    video_path=video_path,
    new_audio_path=mixed_audio_path,
    out_video_path=out_video,
)

out_video


tts_csv: /workspace/baseball_pipeline/data/tts_audio/clip/clip.tts_phrases.with_tts_api.csv | exists: True
[AUDIO] extract_audio: ffmpeg -y -i /workspace/baseball_pipeline/data/input_videos/clip.mp4 -vn -acodec pcm_s16le -ar 48000 -ac 2 /workspace/baseball_pipeline/data/tts_audio/clip/clip.tts_only.orig_tmp.wav


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

[AUDIO] base duration (ms): 160811
[AUDIO] keep_original_gaps=False → 전체 무음 위에 TTS만 올립니다.
[AUDIO] TTS segments: 50 rows
[AUDIO] replace [0.090, 1.000]s utt=clip_1_1 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_1_1.wav
[AUDIO] replace [4.850, 5.580]s utt=clip_1_2 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_1_2.wav
[AUDIO] replace [8.430, 8.700]s utt=clip_1_3 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_1_3.wav
[AUDIO] replace [11.160, 13.550]s utt=clip_2_1 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_2_1.wav
[AUDIO] replace [14.460, 16.690]s utt=clip_2_2 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_2_2.wav
[AUDIO] replace [18.724, 28.674]s utt=clip_3_1 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_3_1.wav
[AUDIO] replace [29.375, 33.965]s utt=clip_4_1 tts=/workspace/baseball_pipeline/data/tts_audio/clip/clip_4_1.wav
[AUDIO] replace [34.875, 35.405]s utt=clip_4_2 tts=/workspace/baseball_pipeline/data/tts_audio/

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

[MUX] saved: /workspace/baseball_pipeline/data/output_videos/clip.tts_only.mp4


[out#0/mp4 @ 0x5aa9a0efc3c0] video:78660kB audio:1315kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.171387%
size=   80112kB time=00:02:40.78 bitrate=4081.6kbits/s speed=  42x    
[aac @ 0x5aa9a0f91180] Qavg: 34055.230


PosixPath('/workspace/baseball_pipeline/data/output_videos/clip.tts_only.mp4')

In [2]:
# [셀: .npy 기반 TTS 배치 테스트 api 사용]
from pathlib import Path

from src import DATA_DIR
from src.tts_fishspeech_npy_api import run_tts_batch_via_npy_server

LLM_OUT_DIR = DATA_DIR / "llm_outputs"

VIDEO_NAME = "clip.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
print("LLM CSV:", llm_csv, "| exists:", llm_csv.exists())

out_csv = run_tts_batch_via_npy_server(
    tts_csv_path=llm_csv,
    api_url="http://127.0.0.1:8080/v1/tts",
    # max_rows=5,    # 먼저 일부만 테스트하고 싶으면 주석 해제
)

out_csv

LLM CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv | exists: True
[TTS_NPY_CLIENT] 입력 CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv
[TTS_NPY_CLIENT] video_stem: clip
[TTS_NPY_CLIENT] 출력 디렉토리: /workspace/baseball_pipeline/data/tts_audio/clip
[TTS_NPY_CLIENT] (0) role=caster voice=caster utt=clip_1_1
[TTS_NPY_CLIENT] Text: 연속 탈삼진
[TTS_NPY_CLIENT] (1) role=caster voice=caster utt=clip_1_2
[TTS_NPY_CLIENT] Text: 풀 카운트 승부
[TTS_NPY_CLIENT] (2) role=caster voice=caster utt=clip_1_3
[TTS_NPY_CLIENT] Text: 볼렛
[TTS_NPY_CLIENT] (3) role=caster voice=caster utt=clip_2_1
[TTS_NPY_CLIENT] Text: 계속되는 풀 카운트 승부 일사주자는 1루
[TTS_NPY_CLIENT] (4) role=caster voice=caster utt=clip_2_2
[TTS_NPY_CLIENT] Text: 바깥쪽 골렛 연속 골렛이 나옵니다.
[TTS_NPY_CLIENT] (5) role=caster voice=caster utt=clip_3_1
[TTS_NPY_CLIENT] Text: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는데 이번 가을은 선취점을 낸 팀이 모두 승리를 한 바 있습니다. 잘 받아친 ...
[TTS_NPY_CLIENT] (6) role=analyst voice=analyst 

KeyboardInterrupt: 

In [3]:
# [셀: Fish-Speech api_server 기반 TTS 배치 wav 넣을떼처럼 되는지 확인]

from pathlib import Path

from src import DATA_DIR
from src.tts_fishspeech_api import run_tts_batch_via_api

LLM_OUT_DIR = DATA_DIR / "llm_outputs"
TTS_REF_DIR = DATA_DIR / "tts_refs"

VIDEO_NAME = "clip.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
caster_ref = TTS_REF_DIR / "caster_prompt.wav"
analyst_ref = TTS_REF_DIR / "analyst_pakchanho_prompt.wav"

print("LLM CSV      :", llm_csv, "| exists:", llm_csv.exists())
print("caster_ref   :", caster_ref, "| exists:", caster_ref.exists())
print("analyst_ref  :", analyst_ref, "| exists:", analyst_ref.exists())

out_csv = run_tts_batch_via_api(
    tts_csv_path=llm_csv,
    caster_ref_wav=caster_ref,
    analyst_ref_wav=analyst_ref,
    api_url="http://127.0.0.1:8080/v1/tts",
    # max_rows=5,  # 먼저 일부만 돌려보고 싶으면 주석 해제
)

out_csv


LLM CSV      : /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv | exists: True
caster_ref   : /workspace/baseball_pipeline/data/tts_refs/caster_prompt.wav | exists: True
analyst_ref  : /workspace/baseball_pipeline/data/tts_refs/analyst_pakchanho_prompt.wav | exists: True
[TTS_API] 입력 CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv
[TTS_API] video_stem: clip
[TTS_API] 출력 디렉토리: /workspace/baseball_pipeline/data/tts_audio/clip
[TTS_API] (0) role=caster voice=caster utt=clip_1_1
[TTS_API] Text: 연속 탈삼진
[TTS_API] (1) role=caster voice=caster utt=clip_1_2
[TTS_API] Text: 풀 카운트 승부
[TTS_API] (2) role=caster voice=caster utt=clip_1_3
[TTS_API] Text: 볼렛
[TTS_API] (3) role=caster voice=caster utt=clip_2_1
[TTS_API] Text: 계속되는 풀 카운트 승부 일사주자는 1루
[TTS_API] (4) role=caster voice=caster utt=clip_2_2
[TTS_API] Text: 바깥쪽 골렛 연속 골렛이 나옵니다.
[TTS_API] (5) role=caster voice=caster utt=clip_3_1
[TTS_API] Text: 테이블 세터진 김지찬, 김성윤 이 두 명의 타자들이 활기를 띠고 있는

PosixPath('/workspace/baseball_pipeline/data/tts_audio/clip/clip.tts_phrases.with_tts_api.csv')

In [2]:
# [셀: TTS 타임라인 오디오 + 영상 합치기]

from pathlib import Path

from src import DATA_DIR  # PROJECT_ROOT 등 이미 정의돼 있다고 가정
from src.audio_timeline import build_tts_audio_timeline, mux_tts_audio_to_video

LLM_OUT_DIR = DATA_DIR / "llm_outputs"
INPUT_VIDEO_DIR = DATA_DIR / "input_videos"

VIDEO_NAME = "clip.mp4"
VIDEO_STEM = Path(VIDEO_NAME).stem

llm_csv = LLM_OUT_DIR / f"{VIDEO_STEM}.tts_phrases.llm_kanana.csv"
tts_audio_dir = DATA_DIR / "tts_audio" / VIDEO_STEM
timeline_wav = tts_audio_dir / f"{VIDEO_STEM}.tts_timeline.wav"

video_path = INPUT_VIDEO_DIR / VIDEO_NAME
out_video_dir = DATA_DIR / "output_videos"
out_video = out_video_dir / f"{VIDEO_STEM}.tts_only.mp4"

print("llm_csv       :", llm_csv, "| exists:", llm_csv.exists())
print("tts_audio_dir :", tts_audio_dir, "| exists:", tts_audio_dir.exists())
print("video_path    :", video_path, "| exists:", video_path.exists())

# 1) start_sec 기준으로 전체 타임라인 오디오 만들기
full_wav = build_tts_audio_timeline(
    llm_csv_path=llm_csv,
    tts_audio_dir=tts_audio_dir,
    out_wav_path=timeline_wav,
    # max_rows=10,  # 먼저 일부만 테스트해보고 싶으면 주석 해제
)

# 2) 원본 비디오(음소거) + 전체 TTS 오디오 붙이기
final_video = mux_tts_audio_to_video(
    video_path=video_path,
    tts_audio_path=full_wav,
    out_video_path=out_video,
)

final_video


llm_csv       : /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv | exists: True
tts_audio_dir : /workspace/baseball_pipeline/data/tts_audio/clip | exists: True
video_path    : /workspace/baseball_pipeline/data/input_videos/clip.mp4 | exists: True
[TIMELINE] LLM CSV: /workspace/baseball_pipeline/data/llm_outputs/clip.tts_phrases.llm_kanana.csv
[TIMELINE] TTS dir : /workspace/baseball_pipeline/data/tts_audio/clip
[TIMELINE] out wav : /workspace/baseball_pipeline/data/tts_audio/clip/clip.tts_timeline.wav
[TIMELINE] sample_rate: 44100
[TIMELINE] add utt=clip_1_1 start_sec=0.090 start_sample=3969 len=59392
[TIMELINE] add utt=clip_1_2 start_sec=4.850 start_sample=213885 len=61440
[TIMELINE] add utt=clip_1_3 start_sec=8.430 start_sample=371763 len=38912
[TIMELINE] add utt=clip_2_1 start_sec=11.160 start_sample=492156 len=141312
[TIMELINE] add utt=clip_2_2 start_sec=14.460 start_sample=637686 len=112640
[TIMELINE] add utt=clip_3_1 start_sec=18.724 start_sample=8257

PosixPath('/workspace/baseball_pipeline/data/output_videos/clip.tts_only.mp4')